# Ejercicio 02: Cálculo de la Matriz TF-IDF y Búsqueda de Consultas en un Corpus

El objetivo de este ejercicio es calcular la matriz TF-IDF de un corpus de documentos y luego aplicar una serie de consultas para recuperar los documentos más relevantes. Este ejercicio te ayudará a comprender cómo funciona el modelo de espacio vectorial y cómo se utiliza TF-IDF para ponderar términos en documentos y consultas.

Seguirás los siguientes pasos:

Pasos del Ejercicio

1. Preprocesamiento del texto:
    * Lectura del corpus desde el archivo TXT.
    * Tokenización de los documentos.
    * Normalización del texto (conversión a minúsculas, eliminación de signos de puntuación).
    * Eliminación de palabras vacías (stopwords).

2. Construcción de la matriz TF-IDF:
    * Cálculo de la frecuencia de término (TF) para cada término en cada documento.
    * Cálculo de la frecuencia inversa de documento (IDF) para cada término en el corpus.
    * Cálculo del peso TF-IDF para cada término en cada documento.

3. Procesamiento de las consultas:
    * Preprocesamiento de las consultas de manera similar a los documentos.
    * Representación de las consultas en el espacio vectorial TF-IDF.

4. Cálculo de similitudes:
    * Cálculo de la similitud entre cada consulta y los documentos del corpus utilizando la similitud del coseno.

5. Ranking de documentos:
    * Ordenar los documentos de mayor a menor similitud para cada consulta.
    * Mostrar los documentos más relevantes para cada consulta.

Consultas

Las consultas a aplicar son las siguientes:

    "inteligencia artificial en medicina"
    "beneficios de la educación a distancia"
    "realidad aumentada en videojuegos"
    "desarrollo personal y hábitos saludables"
    "futuro del comercio electrónico"
    "tecnologías en cine moderno"
    "competencias de e-sports"
    "diagnóstico con dispositivos portátiles"
    "literatura de ciencia ficción"
    "plataformas de streaming"

In [1]:
# Librerias Necesarias
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**1. Preprocesamiento del texto:**

Lectura del corpus desde el archivo TXT.

In [ ]:
# Leer el archivo TXT con el corpus desde la carpeta Datasets
with open('/kaggle/input/02tfidfmatrix-corpus/02tfidfmatrix_corpus.txt', 'r', encoding='utf-8') as file:
    corpus = file.readlines()

# Imprimir el contenido del corpus
print("Contenido del corpus:")
for idx, doc in enumerate(corpus):
    print(f"Documento {idx + 1}: {doc.strip()}")

**Tokenización de los documentos.**

In [ ]:
def tokenizar(text):
    # Usar expresiones regulares para dividir el texto en palabras
    tokens = re.findall(r'\b\w+\b', text.lower())  # Convertimos a minúsculas y extraemos palabras
    return tokens

# Aplicar la tokenización a cada documento en el corpus
corpus_tokenizado = [tokenizar(doc) for doc in corpus]

# Imprimir el corpus tokenizado
print("Corpus tokenizado:")
for idx, tokens in enumerate(corpus_tokenizado):
    print(f"Documento {idx + 1}: {tokens}")

**Normalización del texto (conversión a minúsculas, eliminación de signos de puntuación).**

In [ ]:
def normalizar(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar signos de puntuación utilizando expresiones regulares
    text = re.sub(r'[^\w\s]', '', text)
    
    # Eliminar tildes
    # Crear una tabla de traducción para eliminar tildes
    tabla = str.maketrans(
        "áéíóúüñ",  # caracteres originales
        "aeiouun"   # caracteres reemplazados
    )
    text = text.translate(tabla)
    
    return text

# Aplicar la normalización a cada documento en el corpus
corpus_normalizado = [normalizar(doc) for doc in corpus]

# Imprimir el corpus normalizado
print("Corpus normalizado:")
for idx, doc in enumerate(corpus_normalizado):
    print(f"Documento {idx + 1}: {doc}")

**2. Construcción de la matriz TF-IDF**

Cálculo de la frecuencia de término (TF) para cada término en cada documento.

In [ ]:
# Crear el vectorizador para calcular la frecuencia de términos
vectorizer = CountVectorizer()

# Aplicar el vectorizador al corpus normalizado para obtener la matriz TF
tf_matrix = vectorizer.fit_transform(corpus_normalizado)

# Convertir la matriz a un DataFrame para facilitar la visualización
tf_df = pd.DataFrame(tf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Imprimir la matriz TF
print("Matriz TF (Frecuencia de Términos):")
print(tf_df)

**Visualización de terminos en orden de frecuencia, simplemente para comprobar que los terminos que más se repiten dentro del corpus serian las stopword.**

In [ ]:
# Sumar las frecuencias de cada término en todos los documentos
term_frecuencia = tf_df.sum().sort_values(ascending=False)

# Convertir a un DataFrame para una mejor visualización
term_frecuencia_df = pd.DataFrame(term_frecuencia, columns=['Frecuencia']).reset_index()
term_frecuencia_df.columns = ['Término', 'Frecuencia']

# Imprimir los términos ordenados por frecuencia
print("Términos ordenados por frecuencia:")
print(term_frecuencia_df)

**Cálculo de la frecuencia inversa de documento (IDF) para cada término en el corpus.**

In [ ]:
# Número total de documentos en el corpus
N = len(corpus_normalizado)

# Calcular el número de documentos en los que aparece cada término
doc_frecuencia = (tf_df > 0).sum()

# Calcular el IDF usando la fórmula
idf_values = np.log(N / doc_frecuencia)

# Convertir a un DataFrame para facilitar la visualización
idf_df = pd.DataFrame(idf_values, columns=['IDF']).reset_index()
idf_df.columns = ['Término', 'IDF']

# Imprimir los valores IDF
print("Frecuencia Inversa de Documento (IDF):")
print(idf_df)

In [ ]:
# Ordenar los valores de IDF de mayor a menor
idf_ordenado = idf_df.sort_values(by='IDF', ascending=False)

# Imprimir los valores IDF ordenados
print("Frecuencia Inversa de Documento (IDF) ordenada de mayor a menor:")
print(idf_ordenado)

In [ ]:
# Convertir la matriz TF a un arreglo de NumPy para la multiplicación
tf_array = tf_df.values

# Calcular la matriz TF-IDF multiplicando TF por IDF
tfidf_matriz = tf_array * idf_values.values

# Convertir la matriz TF-IDF a un DataFrame para facilitar la visualización
tfidf_df = pd.DataFrame(tfidf_matriz, columns=idf_df['Término'])

# Imprimir la matriz TF-IDF
print("Matriz TF-IDF:")
print(tfidf_df)

**Procedemos a eliminar las stopwords.**

In [ ]:
stopwords = [
    'la', 'las', 'el', 'los', 'de', 'y', 'en', 'a', 'que', 'con', 'por', 'para', 'como', 'más', 'es', 'se', 'esto', 'este', 
    'esta', 'de', 'un', 'una', 'lo', 'al', 'del', 'su', 'las', 'ser', 'está', 'están', 'o', 'pero', 'no', 'si', 'con', 'su', 
    'sobre', 'entre', 'este', 'también', 'esos', 'esas', 'esto', 'ese', 'por', 'ademas', 'cuando', 'ya', 'nos', 'tiene', 'nosotros', 
    'mi', 'tú', 'te', 'su', 'y', 'se', 'un', 'al', 'desde', 'hasta', 'durante', 'aunque', 'al', 'muy', 'hay', 'donde', 'esto', 
    'ser', 'una', 'los', 'mientras', 'cual', 'quien', 'que', 'sido', 'su', 'sobre', 'esto', 'ella', 'esto', 'yo', 'debería', 
    'su', 'mi', 'tiempo', 'tal', 'vez', 'tan', 'uno', 'los', 'hace', 'muchos', 'otros', 'estas', 'ellas', 'nosotros', 'ellas', 
    'algunas', 'algún', 'sólo', 'vez', 'que', 'más', 'del', 'menos', 'todo', 'al', 'siempre', 'por', 'también', 'en', 'todos'
]

In [ ]:
# Filtrar las columnas de la matriz TF-IDF para eliminar las stopwords en español
columns_to_keep = [term for term in tfidf_df.columns if term not in stopwords]

# Crear una nueva matriz TF-IDF sin las stopwords
tfidf_filtered_df = tfidf_df[columns_to_keep]

# Imprimir la nueva matriz TF-IDF sin stopwords
print("Matriz TF-IDF sin Stopwords:")
print(tfidf_filtered_df)

**3. Procesamiento de las consultas:**

**Preprocesamiento de las consultas de manera similar a los documentos.**

In [ ]:
queries = [
    "inteligencia artificial en medicina",
    "beneficios de la educación a distancia",
    "realidad aumentada en videojuegos",
    "desarrollo personal y hábitos saludables",
    "futuro del comercio electrónico",
    "tecnologías en cine moderno",
    "competencias de e-sports",
    "diagnóstico con dispositivos portátiles",
    "literatura de ciencia ficción",
    "plataformas de streaming"
]

# Aplicar la normalización a cada consulta
queries_normalizadas = [normalizar(query) for query in queries]

# Imprimir las consultas normalizadas
print("Consultas normalizadas:")
for idx, query in enumerate(queries_normalizadas):
    print(f"Consulta {idx + 1}: {query}")

**Representación de las consultas en el espacio vectorial TF-IDF.**

In [ ]:
# Aplicar la normalización y tokenización a las consultas
queries_normalizadas = [normalizar(query) for query in queries]

# Tokenizar las consultas normalizadas
queries_tokenizadas = [tokenizar(query) for query in queries_normalizadas]

# Eliminar las stopwords de las consultas
queries_sin_stopwords = []
for query in queries_tokenizadas:
    query_sin_stopwords = [word for word in query if word not in stopwords]
    queries_sin_stopwords.append(query_sin_stopwords)

# Reconvertir las consultas sin stopwords a texto (para pasarlas al vectorizador)
queries_limpias = [' '.join(query) for query in queries_sin_stopwords]

# Inicializar el vectorizador TF-IDF
vectorizer_tfidf = TfidfVectorizer()

# Convertir la matriz a un DataFrame para facilitar la visualización
queries_tfidf_matrix = vectorizer_tfidf.fit_transform(queries_limpias)
queries_tfidf_df = pd.DataFrame(queries_tfidf_matrix.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

# Imprimir la matriz TF-IDF de las consultas
print("Matriz TF-IDF para las consultas:")
print(queries_tfidf_df)

**4. Cálculo de similitudes:**

Cálculo de la similitud entre cada consulta y los documentos del corpus utilizando la similitud del coseno

In [ ]:
# Inicializar el vectorizador TF-IDF para los documentos
vectorizer_tfidf = TfidfVectorizer(stop_words=stopwords)  # Usamos el mismo vectorizador para consultas y documentos

# Se convierte el corpus de documentos en su representación TF-IDF
tfidf_matrix = vectorizer_tfidf.fit_transform(corpus)

# Transformar las consultas con el mismo vectorizador
queries_tfidf_matrix = vectorizer_tfidf.transform(queries_limpias)

# Calcular la similitud del coseno entre las consultas y los documentos
similitudes_consultas_documentos = cosine_similarity(queries_tfidf_matrix, tfidf_matrix)

# Convertir la matriz de similitudes a un DataFrame para mejor visualización
similitudes_df = pd.DataFrame(similitudes_consultas_documentos,
                              columns=[f'Documento {i+1}' for i in range(len(corpus))],
                              index=[f'Consulta {i+1}' for i in range(len(queries))])

# Imprimir la matriz de similitudes entre consultas y documentos
print("Matriz de Similitud del Coseno entre consultas y documentos:")
print(similitudes_df)

**5. Ranking de documentos:**

Ordenar los documentos de mayor a menor similitud para cada consulta y mostrar los documentos más relevantes.

In [ ]:
for consulta_idx, consulta in similitudes_df.iterrows():
    consulta_idx_int = int(re.search(r'\d+', consulta_idx).group())  # Extrae el número de la consulta
    
    consulta_texto = queries[consulta_idx_int - 1]  # Obtener el texto original de la consulta
    
    # Imprimir el ranking de documentos para cada consulta
    print(f"\nRanking de documentos para la Consulta {consulta_idx_int} - ({consulta_texto}):")
    
    ranking_documentos = consulta.sort_values(ascending=False)  # Ordenamos de mayor a menor
    
    # Mostrar los documentos más relevantes
    top_documentos = ranking_documentos.head(3)  # Los 3 más relevantes
    for doc_idx, score in top_documentos.items():
        # Mostrar el documento y su similitud con el formato deseado
        print(f"**//** {doc_idx}   Similitud Coseno = {score:.6f} **//**")
        
        # Mostrar el texto del documento correspondiente
        doc_texto = corpus[int(doc_idx.split()[-1]) - 1].strip()  # Obtener el texto del documento
        print(f"{doc_texto}")